In [ ]:
import tess_cpm
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astroquery.mast import Tesscut
from astropy.coordinates import SkyCoord
import lightkurve as lk
from IPython.display import HTML
import matplotlib 
matplotlib.rcParams['animation.embed_limit'] = 50

In [ ]:
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams["figure.dpi"] = 400

In [ ]:
# fits_file = "tpf/tess-s0001-4-1_64.525833_-63.615669_64x64_astrocut.fits"
# fits_file = "provided_sources/tess-s0015-1-1_322.878520_47.920650_64x64_astrocut.fits"
# fits_file = "tess-s0002-4-2_83.272000_-69.508733_64x64_astrocut.fits"
# tabby14_file = "ffi/tabby/tess-s0014-2-4_301.564386_44.456886_100x100_astrocut.fits"
tabby15_file = "ffi/tabby/tess-s0015-2-3_301.564386_44.456886_100x100_astrocut.fits"
# tau_ceti_file = "ffi/tau_ceti/tess-s0003-1-2_26.017048_-15.937482_100x100_astrocut.fits"
# sn_file = "supernovae/tess-s0001-4-1_64.525833_-63.615669_100x100_astrocut.fits"
# fits_file = "provided_sources/tess-s0015-1-1_322.878520_47.920650_64x64_astrocut.fits" # interesting eclipsing binary
parenago_file = "provided_sources/tess-s0006-1-1_83.826553_-5.377064_100x100_astrocut.fits"  # Parenago 1925

In [ ]:
tabby15 = tess_cpm.Source(tabby15_file, remove_bad=True)

In [ ]:
tabby15.plot_cutout();

In [ ]:
tabby15.set_aperture(rowlims=[49, 52], collims=[49, 52])
tabby15.plot_cutout(rowlims=[46, 55], collims=[46, 55], show_aperture=True);

In [ ]:
# tabby15.set_aperture(rowrange=[49, 52], colrange=[49, 52])
# tabby15.plot_cutout(rowrange=[47, 53], colrange=[48, 53], show_aperture=True)

In [ ]:
tabby15.plot_pix_by_pix();

In [ ]:
tabby15.add_cpm_model()
tabby15.set_regs([0.01])
times, fluxes, predictions = tabby15.holdout_fit_predict(k=50)

In [ ]:
tabby15.plot_pix_by_pix(data_type="cpm_subtracted_flux", split=True)

In [ ]:
detrended_aperture_lc = tabby15.get_aperture_lc(data_type="cpm_subtracted_flux")

In [ ]:
plt.plot(tabby15.time, detrended_aperture_lc, "-", c="k")
# outliers = tess_cpm.get_outliers(detrended_aperture_lc, window=80, sigma_upper=3, sigma_lower=1.5)
# plt.plot(tabby15.time[outliers], detrended_aperture_lc[outliers], "x", c="red")
# plt.plot(tabby15.time[~outliers], detrended_aperture_lc[~outliers], "-", c="green")

In [ ]:
# tabby15.set_regs([0.0001])
times, fluxes, predictions = tabby15.holdout_fit_predict(k=50, mask=~outliers)

In [ ]:
tabby15.plot_pix_by_pix(data_type="cpm_subtracted_flux")

In [ ]:
with_mask_apt_lc = tabby15.get_aperture_lc(data_type="cpm_subtracted_flux")
plt.plot(tabby15.time, with_mask_apt_lc, "-", c="k")
lc = lk.TessLightCurve(time=tabby15.time, flux=with_mask_apt_lc+1)
# lc.plot()
# plt.plot(tabby15.time, lc.flatten().flux)

In [ ]:
plt.plot(tabby15.time, detrended_aperture_lc)
plt.plot(tabby15.time, with_mask_apt_lc)

In [ ]:
cpm_regs = 10.0 ** np.arange(-6, 6)
min_reg, cdpps = tabby15.calc_min_cpm_reg(cpm_regs, k=50)
print(f"Min CDPP Reg: {min_reg}")

In [ ]:
tabby15.set_regs([min_reg])
tabby15.holdout_fit_predict(k=5)
detrended_apt_lc = tabby15.get_aperture_lc(data_type="cpm_subtracted_lc")
plt.plot(tabby15.time, detrended_apt_lc, "-", c="k")

In [ ]:
tabby15.plot_pix_by_pix(data_type="cpm_subtracted_lc")

In [ ]:
tabby15.set_regs([0.00])
tabby15.holdout_fit_predict(k=tabby15.time.size);
flux_splits = tabby15.get_aperture_lc(data_type="raw", split=True)
splits = tabby15.get_aperture_lc(data_type="cpm_subtracted_lc", split=True)
f, axs = plt.subplots(2, 1, sharex=True, figsize=(12, 8))
for time, split, f_split in zip(tabby15.split_times, splits, flux_splits):
    axs[0].plot(time, split, ".")
    axs[1].plot(time, f_split, ".")

In [ ]:
f, axs = plt.subplots(2, 1, figsize=(12, 8))
for t, s, f_split in zip(tabby15.split_times, splits, flux_splits):
    axs[0].plot(t, s, ".")
    axs[1].plot(t, f_split, ".")

In [ ]:
for t, p, f in zip(tabby15.split_times, splits, flux_splits):
#     plt.plot(t, f, ".")
    plt.plot(t, p, ".")

In [ ]:
# tabby15.rescale()
tabby15.plot_pix_by_pix(data_type="cpm_subtracted_lc")
loo_lc = tabby15.get_aperture_lc(data_type="cpm_subtracted_lc")
plt.plot(tabby15.time, loo_lc, "-", c="k")

In [ ]:
n = tess_cpm.Source(fits_file)
n.plot_cutout(projection="wcs")

In [ ]:
n.set_aperture(rowrange=[29, 32],colrange=[30, 33])
n.plot_cutout(rowrange=[29, 36], colrange=[29, 36], show_aperture=True)

In [ ]:
world = n.cutout_data.wcs_info.wcs_pix2world([[30, 30]], 0)
world
# 322.878520_47.920650
pix = n.cutout_data.wcs_info.wcs_world2pix(world, 0)
pix
print(world)

In [ ]:
n.plot_pix_by_pix()

In [ ]:
n.add_cpm_model()
n.set_regs([0.1])
n.holdout_fit_predict(k=10);

In [ ]:
n.plot_pix_by_pix(data_type="cpm_subtracted_flux", split=True)

In [ ]:
detrended_lc = n.get_aperture_lc(data_type="cpm_subtracted_flux")

In [ ]:
plt.plot(n.time, detrended_lc, "-", c="k")

In [ ]:
lc = lk.TessLightCurve(time=n.time, flux=detrended_lc)
lc.plot()

In [ ]:
pg = lc.to_periodogram(oversample_factor=100)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15, 8))
pg.plot(ax=axs[0], c='k')
pg.plot(ax=axs[1], c='k', view='period')
fig.suptitle("Periodogram", fontsize=20, y=0.95)
period = pg.period_at_max_power
print(f"Max Power Period: {period}")
lc.fold(period.value*2).scatter()
plt.title(f"Folded Lightcurve with Period: {period:.4f}", fontsize=20);

In [ ]:
sn = tess_cpm.Source(sn_file)

In [ ]:
sn.plot_cutout()

In [ ]:
sn.set_aperture(rowrange=[49, 52], colrange=[49, 52])
sn.plot_cutout(show_aperture=True)

In [ ]:
sn.add_cpm_model()
sn.add_poly_model()
sn.set_regs([0.001, 1])
sn.holdout_fit_predict(k=50);

In [ ]:
sn.plot_pix_by_pix(data_type="raw", split=True)

In [ ]:
sn.plot_pix_by_pix(data_type="cpm_subtracted_lc", split=True)

In [ ]:
apt = sn.get_aperture_lc(data_type="cpm_subtracted_lc", split=False)
plt.plot(sn.time, apt)

# Light curve demo (TESS ninja)

In [ ]:
tpf = lk.search_tesscut("AC Pic", sector=13).download(cutout_size=(21, 21))

In [ ]:
mask = tpf.create_threshold_mask(threshold=500)
tpf.plot(aperture_mask=mask)

In [ ]:
lc = tpf.to_lightcurve(aperture_mask=mask)

In [ ]:
lc.plot()

In [ ]:
tpf.interact()

In [ ]:
bgmask = ~tpf.create_threshold_mask(threshold=0)

In [ ]:
tpf.plot(aperture_mask=bgmask)

In [ ]:
regressors = tpf.flux[:, bgmask]

In [ ]:
plt.plot(regressors[:, :10]);

In [ ]:
dm = lk.DesignMatrix(regressors).pca(3)

In [ ]:
dm.plot()

In [ ]:
dm.shape

In [ ]:
corr = lk.RegressionCorrector(lc)
clc = corr.correct(dm)

In [ ]:
clc.plot()

In [ ]:
corr.diagnose()

In [ ]:
lk.designmatrix.create_spline_matrix(lc.time, n_knots=5, degree=2).plot()

In [ ]:
lk.designmatrix.create_spline_matrix()

In [ ]:
clc.plot()

In [ ]:
clc.estimate_cdpp()

In [ ]:
n_clc = clc.normalize()

In [ ]:
n_clc.plot()

In [ ]:
n_clc.estimate_cdpp()

In [ ]:
pg = n_clc.to_periodogram()

In [ ]:
pg.plot(scale="log")

In [ ]:
S3 instead of MASTlc.normalize().interact_bls()

In [ ]:
tpf.interact_sky()

# Parenago 1925 (Eclipsing Binary)

In [ ]:
parenago = tess_cpm.Source(parenago_file)

In [ ]:
fig = parenago.plot_cutout();
# fig.savefig("parenago_cutout.png", dpi=200)

In [ ]:
parenago.set_aperture(rowlims=[51, 57], collims=[47, 50])
# parenago.set_aperture(rowlims=[45, 60], collims=[40, 55])

parenago.plot_cutout(show_aperture=True)

In [ ]:
parenago.plot_pix_by_pix();

In [ ]:
parenago.add_cpm_model(exclusion_size=30)

In [ ]:
parenago.models[2][1].plot_model()
parenago.plot_pix_by_pix();

In [ ]:
parenago.set_regs([0.1])

In [ ]:
parenago.holdout_fit_predict();

In [ ]:
parenago.plot_pix_by_pix(data_type="cpm_subtracted_flux");

In [ ]:
lc = parenago.get_aperture_lc(data_type="cpm_subtracted_flux")

In [ ]:
plt.plot(parenago.time, lc, ".",c="k")
plt.xlabel("Time - 2457000 [Days]", fontsize=30)
plt.ylabel("CPM Flux", fontsize=30)
# plt.tick_params(fontsize=20)
plt.tick_params(labelsize=20)
# plt.savefig("parenago.png", dpi=200)

In [ ]:
outliers = parenago.get_outliers(sigma_lower=0.5)
plt.plot(parenago.time[outliers], lc[outliers])
plt.plot(parenago.time[~outliers], lc[~outliers])

In [ ]:
parenago.holdout_fit_predict(mask=~outliers);

In [ ]:
parenago.plot_pix_by_pix(data_type="cpm_subtracted_flux");
lc = parenago.get_aperture_lc(data_type="cpm_subtracted_flux")
plt.plot(parenago.time, lc, ".",c="k")
plt.xlabel("Time - 2457000 [Days]", fontsize=30)
plt.ylabel("CPM Flux", fontsize=30)
# plt.tick_params(fontsize=20)
plt.tick_params(labelsize=20)
# plt.savefig("parenago.png", dpi=200)

In [ ]:
l = lk.TessLightCurve(time=parenago.time, flux=lc)

In [ ]:
l.plot()

In [ ]:
pg = l.to_periodogram(method="bls", period=np.arange(0.3, 10, 0.001))
fig, axs = plt.subplots(2, 1, figsize=(16, 16))
pg.plot(ax=axs[0], c='k')
pg.plot(ax=axs[1], c='k', view='period')
# fig.suptitle("Periodogram", fontsize=20, y=0.95)
period = pg.period_at_max_power
print(f"Max Power Period: {period}")
l.fold(period.value, t0=1480+period.value/4).scatter()
plt.title(f"Folded Lightcurve with Period: {period:.4f}", fontsize=20)

# Asteroid Detection

In [ ]:
# sd = tess_cpm.Source("provided_sources/tess-s0005-1-4_57.079430_7.641400_100x100_astrocut.fits", remove_bad=True)
sd = tess_cpm.Source("provided_sources/tess-s0005-1-4_57.079430_7.641400_100x100_astrocut.fits", remove_bad=True)

In [ ]:
sd.plot_cutout();

In [ ]:
# sd.set_aperture(rowlims=[30, 70], collims=[30, 70])
# sd.set_aperture(rowlims=[40, 55], collims=[35, 50])
sd.set_aperture(rowlims=[40, 55], collims=[40, 60])
# sd.set_aperture(rowlims=[30, 70], collims=[30, 70])


sd.plot_cutout(show_aperture=True)

In [ ]:
sd.add_cpm_model()
sd.set_regs([0.001])
sd.holdout_fit_predict(k=2);

In [ ]:
# sd.plot_pix_by_pix(data_type="cpm_subtracted_flux");

In [ ]:
ani = sd.make_animation(l=70, h=99, thin=10)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
# ani.save("asteroid.mp4")

In [ ]:
# from astropy.io import fits
# with fits.open("provided_sources/tess-s0005-1-4_57.079430_7.641400_100x100_astrocut.fits", mode="readonly") as hdu:
#     print(hdu[1].header)

In [ ]:
# tic 468